# Big G Express: Predicting Derates
In this project, you will be working with J1939 fault code data and vehicle onboard diagnostic data to try and predict an upcoming full derate. 

J1939 is a communications protocol used in heavy-duty vehicles (like trucks, buses, and construction equipment) to allow different electronic control units (ECUs), like the engine, transmission, and brake systems, to talk to each other. Fault codes in this system follow a standard format so that mechanics and diagnostic tools can understand what's wrong, no matter the make or model.

These fault codes have two parts. First, an SPN (Suspect Parameter Number), which identifies what system or component is having the issue. Second, an FMI (Failure Mode Identifier), which explains how the system is failing (too high, too low, short circuit, etc.).

A derate refers to the truck's computer intentionally reducing engine power or speed to protect itself or force the driver to get it serviced. This is a built-in safety measure. A full derate, the main target in this project, means the vehicle is severely limited, requiring a tow for repairs. Full derates are indicated by an SPN of 5246. 

You have been provided with a two files containing the data you will use to make these predictions (J1939Faults.csv and VehicleDiagnosticOnboardData.csv) as well as two files describing some of the contents (DataInfo.docx and Service Fault Codes_1_0_0_167.xlsx) 

Note that in its raw form the data does not have "labels", so you must define what labels you are going to use and create those labels in your dataset. Also, you will likely need to perform some significant feature engineering in order to build an accurate predictor.

There are service locations at (36.0666667, -86.4347222), (35.5883333, -86.4438888), and (36.1950, -83.174722), so you should remove any records in the vicinity of these locations, as fault codes may be tripped when working on the vehicles.

When evaluating the performance of your model, assume that the cost associated with a missed full derate is approximately $4000 in towing and repairs, and the cost of a false positive prediction is about $500 due to having the truck off the road and serviced unnecessarily. While high accuracy or F1 is nice, we are most interested here in saving the company money, so the final metric to evaulate your model should be the cost savings.

**Project Timeline:**

Thursday, May 8: Present preliminary findings to instructors.
Tuesday, May 13: Present final findings to class.

Your presentation should use slides, not code in a notebook. Your final presentation should include at least the following points:
* What features did you use to predict? Report some of the more impactful features using some kind of feature importance metric.
* If you had used the data prior to 2019 to train your model and had been using it from January 1, 2019 onwards, how many full derates would you have caught? How many false positives? What is the net savings or cost of using your model for that time span? Report your estimate here, even if the model would have been a net negative.

In [1]:
import pandas as pd
from geopy.distance import geodesic
from scipy.spatial import cKDTree
import numpy as np

**Read in data**

In [3]:
faults = pd.read_csv("../data/J1939Faults.csv")
diagnostics = pd.read_csv("../data/VehicleDiagnosticOnboardData.csv")

C:\Users\jdavis\AppData\Local\Temp\ipykernel_26396\1953169697.py:1: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  faults = pd.read_csv("../data/J1939Faults.csv")


**Faults : Cleaning Data**

In [5]:
drop_list = ['ESS_Id', 
             'actionDescription', 
             'ecuSoftwareVersion', 
             'ecuSerialNumber', 
             'ecuModel', 
             'ecuMake', 
             'ecuSource', 
             'faultValue',
             'LocationTimeStamp',
             'MCTNumber']

faults = faults.drop(columns=drop_list)

In [6]:
service_stations = [
    (36.0666667, -86.4347222),
    (35.5883333, -86.4438888),
    (36.1950, -83.174722)
]
def is_near_service_station_kdtree(df, service_stations, threshold_distance=1.0):
    # convert threshold from km to approximate degrees
    # rough approximation: 1 degree ≈ 111 km
    degree_threshold = threshold_distance / 111.0
    
    # KDTree for service stations
    stations_array = np.array(service_stations)
    tree = cKDTree(stations_array)
    
    # query points
    points = np.vstack([df['Latitude'], df['Longitude']]).T
    
    # find points within threshold distance
    # Returns indices of points within degree_threshold of any service station
    indices = tree.query_ball_point(points, degree_threshold)
    
    is_near = np.array([len(idx) > 0 for idx in indices])
    
    return is_near

# define threshold distance
threshold_distance = 1.0 

# apply function
faults['IsServiceStation'] = is_near_service_station_kdtree(
    faults, service_stations, threshold_distance)

In [8]:
threshold_miles = 0.5
threshold_meters = threshold_miles * 1609.34

In [9]:
faults['IsServiceStation'].value_counts(normalize = True)

IsServiceStation
False    0.889795
True     0.110205
Name: proportion, dtype: float64

**Diagnostics (Features) : Cleaning Data**

In [11]:
diagnostics['Value'] = diagnostics['Value'].replace({'FALSE': 0, 'TRUE': 1})

In [12]:
diagnostics_w = diagnostics.pivot(index='FaultId', columns='Name', values='Value')
features = diagnostics_w.reset_index()
features.columns.name = None

**Merged Faults and Features**

In [14]:
combined = pd.merge(faults, 
                    features, 
                    left_on='RecordID', 
                    right_on='FaultId',
                    how = 'left')

**Classifying a Full Derate**

In [16]:
combined['IsDerateFull'] = (combined['spn'] == 5246) & (combined['active'] == True)
combined['IsDerateFull'].value_counts()

IsDerateFull
False    1186728
True         607
Name: count, dtype: int64

In [17]:
faults['IsFullDerate'] = (faults['spn'] == 5246) & (faults['active'] == True)

**Filter Area**

In [19]:
combined_filtered = combined[combined['IsServiceStation'] == False]
combined_filtered = combined_filtered[~((combined_filtered['spn'] == 5246) & (combined_filtered['active'] == False))]
combined_filtered['IsServiceStation'].value_counts(normalize = True)

IsServiceStation
False    1.0
Name: proportion, dtype: float64

**Inspect derate vehicle with highest n faults**

In [27]:
all_derate_vehicles = combined_filtered[(combined_filtered['IsDerateFull'] == True)]
grouped_derate_vehicles = all_derate_vehicles.groupby('EquipmentID').size().reset_index(name='row_count')
grouped_derate_vehicles = grouped_derate_vehicles.sort_values(by='row_count', ascending=False)
print('EquipmentID 1524 has the most rows to inspect')
grouped_derate_vehicles.head()

EquipmentID 1524 has the most rows to inspect


,EquipmentID,row_count
38,1524,31
43,1535,23
39,1525,15
45,1539,14
3,305,13


**Add a New Feature: Severity Level**

In [29]:
import re

def extract_severity(text):

    if pd.isna(text):
        return np.nan
        
    # "Severity" followed by "Low", "Medium", or "High"
    pattern = r'Severity\s+(Low|Medium|High)'
    
    # Search for the pattern 
    match = re.search(pattern, text)
    
    if match:
        # Return "Severity" plus the matched level
        return f"Severity {match.group(1)}"
    else:
        return np.nan

combined_filtered['SeverityLevel'] = combined_filtered['eventDescription'].apply(extract_severity)

In [30]:
severity_map = {
    'Severity Low': 1,
    'Severity Medium': 2,
    'Severity High': 3
}

combined_filtered['SeverityLevelFeature'] = combined_filtered['SeverityLevel'].map(severity_map)

combined_filtered.loc[combined_filtered['spn'] == 1569, 'SeverityLevelFeature'] = 4

In [31]:
inspect_column = combined_filtered[['eventDescription', 'SeverityLevel','SeverityLevelFeature']].dropna(subset=['SeverityLevel'])
inspect_column.head()

,eventDescription,SeverityLevel,SeverityLevelFeature
0,Low (Severity Low) Engine Coolant Level,Severity Low,1.0
5,Low (Severity Low) Engine Coolant Level,Severity Low,1.0
6,Low (Severity Low) Engine Coolant Level,Severity Low,1.0
7,Low (Severity Low) Engine Coolant Level,Severity Low,1.0
8,High (Severity Low) Water In Fuel Indicator,Severity Low,1.0


**Convert Features to_numeric**

In [42]:
feature_numeric_columns = [
    'AcceleratorPedal', 'BarometricPressure', 'CruiseControlSetSpeed', 
    'DistanceLtd', 'EngineCoolantTemperature', 'EngineLoad', 'EngineOilPressure', 'EngineOilTemperature',
    'EngineRpm', 'EngineTimeLtd', 'FuelLevel', 'FuelLtd', 'FuelRate', 'FuelTemperature', 
    'IntakeManifoldTemperature', 'ParkingBrake', 'ServiceDistance', 'Speed', 
    'SwitchedBatteryVoltage', 'Throttle', 'TurboBoostPressure'
]

combined_filtered[feature_numeric_columns] = combined_filtered[feature_numeric_columns].apply(pd.to_numeric, errors='coerce')

**Identify derate rows before creating derate window**

In [44]:
inspect_derate_rows = combined_filtered[combined_filtered['IsDerateFull'] == True][['IsDerateFull', 'active']].value_counts()
inspect_derate_rows

IsDerateFull  active
True          True      498
Name: count, dtype: int64

**Identify False Derates : any subsequent derate within 24hr period**

In [47]:
combined_filtered['EventTimeStamp'] = pd.to_datetime(combined_filtered['EventTimeStamp'])
combined_filtered['IsDerateActual'] = (combined_filtered['spn'] == 5246)

# filter to only derate events and sort by equipment ID and timestamp
derate_events = combined_filtered[combined_filtered['spn'] == 5246].copy()
derate_events = derate_events.sort_values(['EquipmentID', 'EventTimeStamp'])

# empty list to store indices of duplicate derates (within 24 hours)
duplicate_indices = []

# group by equipment id
for equipment_id, group in derate_events.groupby('EquipmentID'):
    # Reset the index for easier iteration
    group = group.reset_index()
    
    # keep track of the last valid derate timestamp
    last_valid_timestamp = None
    
    for i, row in group.iterrows():
        current_timestamp = row['EventTimeStamp']
        
        if last_valid_timestamp is None:
            # First derate for this equipment - keep it
            last_valid_timestamp = current_timestamp
        elif (current_timestamp - last_valid_timestamp).total_seconds() < 24 * 3600:
            # This derate is within 24 hours of the last valid one - mark as duplicate
            duplicate_indices.append(row['index'])
        else:
            # This derate is more than 24 hours after the last valid one - keep it
            last_valid_timestamp = current_timestamp

# mark duplicates as not actual derates
if duplicate_indices:
    combined_filtered.loc[duplicate_indices, 'IsDerateActual'] = False

# results
verification_df = combined_filtered[combined_filtered['spn'] == 5246][['EventTimeStamp', 'EquipmentID', 'spn', 'IsDerateActual']].sort_values(['EquipmentID', 'EventTimeStamp'])

# verify
verification_df['TimeSincePrevDerate'] = verification_df.groupby('EquipmentID')['EventTimeStamp'].diff()
verification_df.head()

,EventTimeStamp,EquipmentID,spn,IsDerateActual,TimeSincePrevDerate
516208,2016-07-12 19:11:07,301,5246,True,NaT
1171245,2020-01-06 10:13:57,302,5246,True,NaT
1173036,2020-01-13 13:18:31,302,5246,True,7 days 03:04:34
1181996,2020-02-14 11:21:54,302,5246,True,31 days 22:03:23
376483,2016-02-15 10:59:28,304,5246,True,NaT


In [49]:
combined_filtered['IsDerateActual'].value_counts()

IsDerateActual
False    1055692
True         355
Name: count, dtype: int64

**Filter our False Derates**

In [51]:
combined_filtered = combined_filtered[~((combined_filtered['spn'] == 5246) & (combined_filtered['IsDerateActual'] == False))]

**Derate Window**

In [53]:
# datetime format
combined_filtered['EventTimeStamp'] = pd.to_datetime(combined_filtered['EventTimeStamp'])

# sort
combined_filtered = combined_filtered.sort_values(['EquipmentID', 'EventTimeStamp'])

# intitial target column
combined_filtered['DeratePredictionTarget'] = 0

# helper dataframe with just the derate events
derate_events = combined_filtered[combined_filtered['IsDerateActual']].copy()

# group by EquipmentID 
for equipment_id, group in combined_filtered.groupby('EquipmentID'):
    # Get derate events for this truck only
    truck_derates = derate_events[derate_events['EquipmentID'] == equipment_id]
    
    if len(truck_derates) > 0:
        # get indices and timestamps for this truck's rows
        truck_indices = group.index
        truck_timestamps = group['EventTimeStamp'].values
        
        # for each derate event in this truck
        for _, derate_row in truck_derates.iterrows():
            derate_time = derate_row['EventTimeStamp']
            
            # window: 
            window_start = derate_time - pd.Timedelta(hours=8)
            window_end = derate_time - pd.Timedelta(hours=.001)
            
            # all events in the prediction window
            in_window = (truck_timestamps >= window_start) & (truck_timestamps <= window_end)
            indices_to_mark = truck_indices[in_window]
            
            # marked as predicting a derate
            combined_filtered.loc[indices_to_mark, 'DeratePredictionTarget'] = 1

# results
print(f"Total events: {len(combined_filtered)}")
print(f"Events predicting a derate: {combined_filtered['DeratePredictionTarget'].sum()}")

Total events: 1055904
Events predicting a derate: 1090


**Identify features for model:**

*Are there features that show trends over time?*

In [54]:
features = [
    'DistanceLtd',
    'EngineOilTemperature',
    'TurboBoostPressure',
    'FuelRate',
    'EngineLoad',
    'EngineOilPressure',
    'EngineCoolantTemperature',
    'BarometricPressure',
    'EngineRpm',
    'IntakeManifoldTemperature',
    'FuelTemperature',
    'SwitchedBatteryVoltage',
    'SeverityLevelFeature'
]

**Statistic Models**

In [57]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE

**Separate Train and Test Data**

In [59]:
combined_filtered['EventTimeStamp'] = pd.to_datetime(combined['EventTimeStamp'], errors='coerce')
combined_filtered['IsTestData'] = combined_filtered['EventTimeStamp'] >= '2019-01-01'
combined_filtered['IsTestData'].value_counts(normalize=True)

IsTestData
False    0.894563
True     0.105437
Name: proportion, dtype: float64

**Random Forest**

In [123]:
# split data into train and test sets
train_data = combined_filtered[combined_filtered['IsTestData'] == False]
test_data = combined_filtered[combined_filtered['IsTestData'] == True]

# prepare features and target
X_train = train_data[features]
y_train = train_data['DeratePredictionTarget']
X_test = test_data[features]
y_test = test_data['DeratePredictionTarget']

# numeric conversion and missing values
X_train = X_train.apply(pd.to_numeric, errors='coerce')
X_test = X_test.apply(pd.to_numeric, errors='coerce')
X_train = X_train.fillna(X_train.mean())
X_test = X_test.fillna(X_test.mean())

# scale
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# OPTIMIZATION 1
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=42)
X_train_resampled, y_train_resampled = ros.fit_resample(X_train_scaled, y_train)

# OPTIMIZATION 2
random_forest_model = RandomForestClassifier(
    n_estimators=10,     
    max_depth=30,        
    min_samples_split=30, 
    n_jobs=-1,           
    random_state=42,
    class_weight='balanced'
)

# train the model
random_forest_model.fit(X_train_resampled, y_train_resampled)

# predictions
y_pred = random_forest_model.predict(X_test_scaled)

# evaluate
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

# feature importance
feature_importances = pd.DataFrame({
    'Feature': features,
    'Importance': random_forest_model.feature_importances_
}).sort_values('Importance', ascending=False)
print("\nTop 10 Important Features:")
print(feature_importances.head(10))

Accuracy: 0.9983
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00    111148
           1       0.33      0.01      0.01       183

    accuracy                           1.00    111331
   macro avg       0.67      0.50      0.50    111331
weighted avg       1.00      1.00      1.00    111331

Confusion Matrix:
 [[111146      2]
 [   182      1]]

Top 10 Important Features:
                      Feature  Importance
12       SeverityLevelFeature    0.186219
0                 DistanceLtd    0.138600
1        EngineOilTemperature    0.083529
5           EngineOilPressure    0.079649
9   IntakeManifoldTemperature    0.077450
8                   EngineRpm    0.069343
10            FuelTemperature    0.068065
6    EngineCoolantTemperature    0.064363
7          BarometricPressure    0.055468
3                    FuelRate    0.054940


**Confusion Matrix**

In [125]:
cm = confusion_matrix(y_test, y_pred)

TN = cm[0, 0]  
FP = cm[0, 1] 
FN = cm[1, 0] 
TP = cm[1, 1]  
#TP = 12

In [127]:
Costs = (FP * 500)
Savings = (TP * 4000)


Net = Savings - Costs
print("Model Net Savings Results")
print(f"True Negatives: {TN}")
print(f"False Positives: {FP}")
print(f"False Negatives: {FN}")
print(f"True Positives: {TP}")
print(f"Money Saved: ${Net:,.2f}")

Model Net Savings Results
True Negatives: 111146
False Positives: 2
False Negatives: 182
True Positives: 1
Money Saved: $3,000.00


In [ ]:
test_data['Predicted'] = y_pred

test_data['Correct_Prediction'] = test_data['DeratePredictionTarget'] == test_data['Predicted']

test_data.head()

columns_to_view = ['DeratePredictionTarget', 'Predicted', 'Correct_Prediction'] + features[:5]  
test_data[columns_to_view].head()

misclassified = test_data[test_data['DeratePredictionTarget'] != test_data['Predicted']]